In [9]:
data = '''???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1'''

In [35]:
import re
import itertools
from copy import copy

In [155]:
def is_valid(spl, pat):
    #print('  ', [ len(x) for x in ''.join(spl).split('.') ], pat)
    return [ len(x) for x in ''.join(spl).split('.') ] == pat
    
for line in data.splitlines():
    sp, arr = line.split(' ')
    spl = list(sp)
    arr = [ int(x) for x in arr.split(',') ]

    output = []
    print('for', sp)
    # match any we have
    '''
    haves = []
    carr = []
    for i in arr:
        if m := re.search(r'(?<!#)#{%s}(?!#)' % i, sp):
            haves.append(i)
            print(f"we know we have {i} at {m.span()}")
        else:
            carr.append(i)
    print("we want to find", carr
    '''
    total = 0

    matches = list(re.finditer(r'\?+', sp))
    print(matches)

    for m in re.finditer(r'\?+', sp):
        s = len(m.group())
        for p in itertools.product(*([['#','.']]*s)):
            np = copy(spl)
            #print(m)
            x,y = m.span()
            np[x:y] = p
            print('  made', ''.join(np), is_valid(np, arr))
            if is_valid(np, arr):
                total += 1
    print(total)
    continue
        
    for i in carr:
        
        break
        for p in itertools.product(['#','.'], repeat=spl.count("?")):
            print(p)
            working = copy(spl)
            p_list=list(p)
            for i, x in enumerate(working):
                if x == "?":
                    working[i] = p_list.pop()

        break
        
        output.append(working)
    for o in output:
        print(o)
    
    #print(sp)
    #print(arr)
    
    break

for ???.###
[<re.Match object; span=(0, 3), match='???'>]
  made ###.### False
  made ##..### False
  made #.#.### True
  made #...### False
  made .##.### False
  made .#..### False
  made ..#.### False
  made ....### False
1
for .??..??...?##.
[<re.Match object; span=(1, 3), match='??'>, <re.Match object; span=(5, 7), match='??'>, <re.Match object; span=(10, 11), match='?'>]
  made .##..??...?##. False
  made .#...??...?##. False
  made ..#..??...?##. False
  made .....??...?##. False
  made .??..##...?##. False
  made .??..#....?##. False
  made .??...#...?##. False
  made .??.......?##. False
  made .??..??...###. False
  made .??..??....##. False
0
for ?#?#?#?#?#?#?#?
[<re.Match object; span=(0, 1), match='?'>, <re.Match object; span=(2, 3), match='?'>, <re.Match object; span=(4, 5), match='?'>, <re.Match object; span=(6, 7), match='?'>, <re.Match object; span=(8, 9), match='?'>, <re.Match object; span=(10, 11), match='?'>, <re.Match object; span=(12, 13), match='?'>, <re.Match ob

In [133]:
[ len(x) for x in ''.join(['#', '#', '#', '.', '#', '#', '#']).split('.') ]

[3, 3]